# SparkSQL
01_ 개념과 워드카운트 예제
<div style="text-align: right">
2018.05.12 / 발제자 김연진
</div>

<br>
### Reference
- 빅데이터 분석을 위한 스파크2 프로그래밍, 백성민, 위키북스
- Spark SQL, DataFrames and Datasets Guide, https://spark.apache.org/docs/latest/sql-programming-guide.html

<hr>
### 순서
- 스파크 SQL
- 데이터셋
- 연산의 종류
- 예제: 워드카운트

<hr>

## 스파크 SQL
- RDD에서는 '스키마' 정보를 표현할 수 없음 ==> 보완하기 위해 등장
- 데이터를 다루는 방법
    - SQL을 사용
    - 데이터셋(Dataset) API를 이용
<br><br>

## 데이터셋
- 데이터 모델이자 API
- RDD와 마찬가지로 분산 오브젝트 컬렉션에 대한 프로그래밍 모델
- 트랜스포메이션과 액션 연산을 포함
- RDD에서 제공하던 형태의 연산 + 기존 데이터프레임 같은 형태의 연산 사용 가능!
- 참고: 데이터 프레임 = org.apache.spark.sql.Row 타입의 객체로 구성된 데이터셋
<br><br>

## 연산의 종류와 주요 API
- **트랜스포메이션**: 데이터셋을 생성
    - 타입 연산 (Typed Operations)
    - 비타입 연산 (Untyped Operations, 데이터 프레임을 통한 연산)
    
    
    
- **액션**: 실제 데이터 처리를 수행하고 결과를 생성

<div style="text-align: right">
※ 파이썬의 경우 데이터셋 API를 지원하지 않으므로 기존의 데이터프레임만 사용 가능
</div>
<br><br>

## 예제: 워드카운트

~~~
<코드의 순서>
1. 스파크 세션 생성
2. 스파크 세션으로부터 데이터셋/데이터 프레임 생성
3. 생성된 데이터셋/데이터프레임을 이용해 데이터 처리
4. 처리된 결과 데이터를 외부 저장소에 저장
5. 스파크세션 종료
~~~

#### 1. 스파크 세션 생성

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession\
          .builder\
          .appName("Sample")\
          .master("local[*]")\
          .getOrCreate()

#### 2. 스파크 세션으로부터 데이터셋/데이터 프레임 생성

In [4]:
source = "file:///home/ubuntu/data/2018-1/chapter05/spark05_sparkSQL/resources/countMe.txt"         # hdfs 상의 파일이나 md 파일도 가능
df = spark.read.text(source)               # 데이터소스로 텍스트 파일을 지정했으므로 text()메소드 사용

#### 3. 생성된 데이터셋/데이터프레임을 이용해 데이터 처리

In [5]:
wordDF = df.select(explode(split(col("value"), " ")).alias("word"))
result = wordDF.groupBy("word").count()
result.show()

+----------+-----+
|      word|count|
+----------+-----+
|      July|    1|
|     those|    5|
|     spoil|    3|
|    travel|    2|
|       few|    1|
|pack-drill|    1|
|    waters|    1|
|    harder|    1|
|      hope|    1|
|      some|    1|
|    taking|    1|
|   Sabbath|    1|
|     parts|    1|
|      lies|    1|
|    Mighty|    1|
|  Tomorrow|    2|
|   vinegar|    1|
|   stomach|    2|
|   showers|    2|
|   flowers|    2|
+----------+-----+
only showing top 20 rows



#### 4. 처리된 결과 데이터를 외부 저장소에 저장

In [14]:
outdir = "file:///home/ubuntu/data/2018-1/chapter05/spark05_sparkSQL/output2/"
result.write.format("csv").save(outdir)

#### 5. 스파크세션 종료

In [15]:
spark.stop()